# Task 4 - Optimize Portfolio Based on Forecast

### 1. Compute Expected Returns

TSLA: Use the forecasted returns from Task 3 (e.g., average daily return from the 12-month forecast).
BND, SPY: Use historical average daily returns (annualized) from the full dataset (2016-01-01 to 2025-07-31).

In [ ]:
import pandas as pd
import numpy as np
from pypfopt import EfficientFrontier, risk_models, expected_returns
import matplotlib.pyplot as plt

# Load historical data
tsla_close = pd.read_csv('data/TSLA_historical.csv', index_col='Date', parse_dates=True)['Adj Close']
bnd_close = pd.read_csv('data/BND_historical.csv', index_col='Date', parse_dates=True)['Adj Close']
spy_close = pd.read_csv('data/SPY_historical.csv', index_col='Date', parse_dates=True)['Adj Close']
prices = pd.DataFrame({'TSLA': tsla_close, 'BND': bnd_close, 'SPY': spy_close}).dropna()

# Historical daily returns
historical_returns = prices.pct_change().dropna()

# Load TSLA forecast (from Task 3, e.g., forecast_df['Forecast'])
forecast_df = pd.DataFrame({'Forecast': lstm_forecast}, index=future_dates)  # Adjust if using SARIMA
forecast_returns = forecast_df['Forecast'].pct_change().dropna()
tsla_expected_return = forecast_returns.mean() * 252  # Annualize

# Historical annualized returns for BND, SPY
bnd_expected_return = historical_returns['BND'].mean() * 252
spy_expected_return = historical_returns['SPY'].mean() * 252

# Expected returns vector
expected_returns_vec = np.array([tsla_expected_return, bnd_expected_return, spy_expected_return])
print("Expected Annual Returns:", dict(zip(['TSLA', 'BND', 'SPY'], expected_returns_vec)))

### 2. Compute Covariance Matrix

In [ ]:
# Covariance matrix (annualized)
cov_matrix = risk_models.sample_cov(historical_returns, returns_data=True, frequency=252)
print("Covariance Matrix:\n", cov_matrix)

### 3. Generate Efficient Frontier

In [ ]:
# Initialize Efficient Frontier
ef = EfficientFrontier(expected_returns_vec, cov_matrix, weight_bounds=(0, 1))

# Maximum Sharpe Ratio Portfolio
sharpe_weights = ef.max_sharpe(risk_free_rate=0.02)  # Assume 2% risk-free rate
sharpe_perf = ef.portfolio_performance(verbose=True, risk_free_rate=0.02)

# Minimum Volatility Portfolio
min_vol_weights = ef.min_volatility()
min_vol_perf = ef.portfolio_performance(verbose=True, risk_free_rate=0.02)

# Generate points for Efficient Frontier
ef_points = []
returns_range = np.linspace(min_vol_perf[0], sharpe_perf[0] + 0.02, 50)
for ret in returns_range:
    ef = EfficientFrontier(expected_returns_vec, cov_matrix)
    ef.efficient_return(target_return=ret)
    ret, vol, _ = ef.portfolio_performance()
    ef_points.append((vol, ret))

# Plot Efficient Frontier
vols, rets = zip(*ef_points)
plt.figure(figsize=(10, 6))
plt.scatter(vols, rets, c='blue', marker='.', label='Efficient Frontier')
plt.scatter(sharpe_perf[1], sharpe_perf[0], c='red', marker='*', s=200, label='Max Sharpe Portfolio')
plt.scatter(min_vol_perf[1], min_vol_perf[0], c='green', marker='*', s=200, label='Min Volatility Portfolio')
plt.xlabel('Annualized Volatility')
plt.ylabel('Annualized Return')
plt.title('Efficient Frontier for TSLA, BND, SPY Portfolio')
plt.legend()
plt.savefig('reports/efficient_frontier.png')
plt.show()